In [1]:
from InteractionDetection import *
from sklearn.linear_model import LinearRegression
from Metrics import *
from TimewiseKfold import *
import numpy as np

In [2]:
scaled_folds, PROVINCES_NUM = splitting_data(is_scaled=True, region=2)

In [3]:
selected_features = ['Rdays_VII', 'Rdays_X', 'Srad_I', 'Srad_IX', 'Srad_X']

#1st iteration
selected_features += ['Eva_I', 'Rdays_III', 'Tmax_avg_IX', 'Tmin_II']

#2st iteration
selected_features += ['Eva_IV']

#3rd iteration
selected_features += ['Prcp_max_XII']

#4th iteration
selected_features += ['Prcp_X']

alphas_res = [
   np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0. , 0.5, 0. , 0.5, 0.5]), 
   np.array([0.5, 0. , 0.5, 0.5, 0.5, 0.5, 0.5, 0. , 0.5, 0.5, 0.5, 0.5]), 
   np.array([0.5, 0. , 0.5, 0. , 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]), 
   np.array([0.5, 0.5, 0.5, 0.5, 0. , 0.5, 0.5, 0.5, 0.5, 1. , 0.5, 0.5]), 
   np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1. , 0. , 0.5, 0.5]), 
   np.array([0. , 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1. , 0.5, 0.5]), 
   np.array([0.5, 1. , 0.5, 0.5, 0.5, 0.5, 0. , 0.5, 0.5, 0.5, 0.5, 0.5]) 
]

deltas_res = [
   np.array([0., 0., 0., 0., 1., 0.]), 
   np.array([0., 0., 1., 0., 0., 0.]), 
   np.array([0., 0., 0., 1., 0., 0.]), 
   np.array([1., 0., 0., 0., 0., 0.]), 
   np.array([0., 0., 0., 0., 1., 0.]), 
   np.array([0., 0., 0., 0., 1., 0.]), 
   np.array([0., 1., 0., 0., 0., 0.])
]
 
transformed_folds = transform_data(scaled_folds=scaled_folds, selected_features=selected_features, alphas=alphas_res, deltas=deltas_res)

rmse_train, rmse_validatioin = RMSE_FOLDS(transformed_folds=transformed_folds, PROVINCES_NUM=PROVINCES_NUM)

print(rmse_train, rmse_validatioin)

coef = []

table = metrics_table()

for (i, fold) in enumerate(transformed_folds):
    X_train, X_test, y_train, y_test = split_train_test(fold=fold)
    
    linear_model = LinearRegression(n_jobs=-1).fit(X=X_train, y=y_train)
    
    row = [MAXIMUM_YEAR - FOLD_NUM + 1 + i]
    
    row += metrics_calculation(y_actual=y_test, y_pred=linear_model.predict(X_test), y_train=y_train)
    
    table.loc[len(table)] = row
    
    del linear_model
    
table

2.093475160828057 1.1998343396018514


,Year,RMSE,R2_Score,MSE,MAE,RAE,RSE
0,2018.0,1.243549,0.476394,1.546415,0.856808,0.597661,0.569677
1,2019.0,1.129843,0.197289,1.276546,0.927718,0.898089,0.806955
2,2020.0,5.939001,0.017396,35.271732,3.854420,1.037715,1.377431
